# Preprocessing 

책소개, 목차가 없으면 제거

In [1]:
import pandas as pd
import numpy as np
import os # 파일 목록을 처리해야 할 경우를 대비해 import 합니다.

def preprocess_book_data(file_path):
    """
    주어진 CSV 파일에서 '책소개' 또는 '목차' 컬럼이 둘 다 결측치인 행을 제거하는 함수입니다.

    Args:
        file_path (str): 처리할 CSV 파일의 경로.

    Returns:
        pd.DataFrame: 전처리된 데이터프레임.
    """
    try:
        # 한국어 인코딩 문제로 'utf-8', 'euc-kr', 'cp949' 순으로 시도하여 파일을 읽어옵니다.
        try:
            df = pd.read_csv(file_path, encoding='utf-8')
        except UnicodeDecodeError:
            try:
                df = pd.read_csv(file_path, encoding='euc-kr')
            except UnicodeDecodeError:
                df = pd.read_csv(file_path, encoding='cp949')

        print(f"--- 파일: {os.path.basename(file_path)} ---")
        print("초기 데이터프레임 행 수:", df.shape[0])

        # '책소개' 또는 '목차' 컬럼 중 하나라도 결측치가 아닌(notna) 행만 선택합니다.
        # '|' (or) 연산자를 사용하여 둘 중 하나라도 True이면 해당 행을 유지합니다.
        # .notna()는 결측치(NaN)가 아닌 값을 True로 반환합니다.
        cleaned_df = df[df['책소개'].notna() | df['목차'].notna()].copy()
        
        # float에서 int로 변환 (SettingWithCopyWarning 방지)
        cleaned_df['ISBN13'] = cleaned_df['ISBN13'].fillna(0).astype(int)
        cleaned_df['순번/순위'] = cleaned_df['순번/순위'].fillna(0).astype(int)
        cleaned_df['출간일'] = cleaned_df['출간일'].fillna(0).astype(int)
        cleaned_df['정가'] = cleaned_df['정가'].fillna(0).astype(int)
        cleaned_df['판매가'] = cleaned_df['판매가'].fillna(0).astype(int)
        cleaned_df['세일즈포인트'] = cleaned_df['세일즈포인트'].fillna(0).astype(int)

        # ItemId, 부가기호, 마일리지 컬럼 제거
        cleaned_df = cleaned_df.drop(columns=['ItemId', '부가기호', '마일리지'])


        print("제거된 행 수:", df.shape[0] - cleaned_df.shape[0])
        print("전처리된 데이터프레임 행 수:", cleaned_df.shape[0])
        print("-" * 30)

        return cleaned_df

    except FileNotFoundError:
        print(f"오류: 파일을 찾을 수 없습니다: {file_path}")
        return pd.DataFrame()
    except Exception as e:
        print(f"처리 중 오류 발생: {e}")
        return pd.DataFrame()

# --- 사용 예시 ---
base_path = './data'  # 데이터 파일이 위치한 디렉토리 경로
categories = ['대학교재_전문서적', '만화', '사회과학', '소설_시_희곡', '어린이', '에세이', '여행', '역사', 
              '예술_대중문화', '요리_살림', '유아', '인문학', '자기계발', '장르소설', '종교_역학', '좋은부모', '청소년']

combined = pd.DataFrame()

# 모든 카테고리에 대해 전처리 수행
for category in categories:
    file_path = os.path.join(base_path, 'csv', f'{category}.csv')
    cleaned_df = preprocess_book_data(file_path)
    combined = pd.concat([combined, cleaned_df], ignore_index=True)

# 전처리된 전체 데이터프레임을 하나의 CSV 파일로 저장
output_path = os.path.join(base_path, 'combined.csv')
combined.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"전처리된 데이터가 '{output_path}'에 저장되었습니다.")

--- 파일: 대학교재_전문서적.csv ---
초기 데이터프레임 행 수: 200
제거된 행 수: 16
전처리된 데이터프레임 행 수: 184
------------------------------
--- 파일: 만화.csv ---
초기 데이터프레임 행 수: 200
제거된 행 수: 71
전처리된 데이터프레임 행 수: 129
------------------------------
--- 파일: 사회과학.csv ---
초기 데이터프레임 행 수: 999
제거된 행 수: 799
전처리된 데이터프레임 행 수: 200
------------------------------
--- 파일: 소설_시_희곡.csv ---
초기 데이터프레임 행 수: 999
제거된 행 수: 800
전처리된 데이터프레임 행 수: 199
------------------------------
--- 파일: 어린이.csv ---
초기 데이터프레임 행 수: 999
제거된 행 수: 802
전처리된 데이터프레임 행 수: 197
------------------------------
--- 파일: 에세이.csv ---
초기 데이터프레임 행 수: 999
제거된 행 수: 803
전처리된 데이터프레임 행 수: 196
------------------------------
--- 파일: 여행.csv ---
초기 데이터프레임 행 수: 999
제거된 행 수: 801
전처리된 데이터프레임 행 수: 198
------------------------------
--- 파일: 역사.csv ---
초기 데이터프레임 행 수: 999
제거된 행 수: 799
전처리된 데이터프레임 행 수: 200
------------------------------
--- 파일: 예술_대중문화.csv ---
초기 데이터프레임 행 수: 999
제거된 행 수: 800
전처리된 데이터프레임 행 수: 199
------------------------------
--- 파일: 요리_살림.csv ---
초기 데이터프레임 행 수: 200
제거된 행 수: 3
전처

# 특수문자 및 필요없는 문자 제거

### 함수들

In [7]:
import re
import pandas as pd

def clean_text(text, patterns_to_remove):
    """
    텍스트에서 특정 패턴과 단어를 제거하는 함수입니다.
    
    Args:
        text (str): 정제할 텍스트
        patterns_to_remove (list): 제거할 패턴 또는 단어의 리스트
        
    Returns:
        str: 정제된 텍스트
    """
    if pd.isna(text):
        return text
    
    text = str(text)
    for pattern in patterns_to_remove:
        text = re.sub(pattern, '', text)
    
    return text.strip()

In [8]:
def replace_chars(text, replacement_dict):
    """
    텍스트에서 특정 문자를 다른 문자로 대체하는 함수입니다.
    
    Args:
        text (str): 대체할 텍스트
        replacement_dict (dict): {'기존문자': '새로운문자'} 형태의 딕셔너리
        
    Returns:
        str: 대체된 텍스트
    
    Example:
        replacement_dict = {'–': '-', '—': '-', '…': '...'}
        text = "안녕–세상–반갑습니다"
        result = replace_chars(text, replacement_dict)
        # 결과: "안녕-세상-반갑습니다"
    """
    if pd.isna(text):
        return text
    
    text = str(text)
    for old_char, new_char in replacement_dict.items():
        text = text.replace(old_char, new_char)
    
    return text

In [9]:
# 특수문자 분석
def find_special_chars(text, specialChars):
    """텍스트에서 특수문자 찾기"""
    if pd.isna(text):
        return set()
    text = str(text)
    # 알파벳, 숫자, 한글, 공백을 제외한 모든 문자를 특수문자로 간주
    special = re.findall(specialChars, text, re.UNICODE)
    # special = re.findall(r'[^\w\s가-힣\t!?()\'\"-\./,\[\]:;]', text, re.UNICODE)
    return set(special)

## 특수문자 분석하기

In [10]:
file_path = os.path.join(base_path, 'combined_preprocessed.csv')
df = pd.read_csv(file_path, encoding='utf-8')
# 모든 텍스트 컬럼에서 특수문자 수집
all_special_chars = set()
special_char_by_column = {}
exclude_specials = r'[^\w\s가-힣\t!?()\'\"-\./,\[\]:;<>=@~|]'

for col in df.columns:
    if df[col].dtype == 'object' and col != '상품명':  # 텍스트 컬럼만
        col_special_chars = set()
        for text in df[col]:
            col_special_chars.update(find_special_chars(text, exclude_specials))
        special_char_by_column[col] = col_special_chars
        all_special_chars.update(col_special_chars)

print(f"발견된 특수문자 (총 {len(all_special_chars)}개):")
print(f"sorted(all_special_chars): {sorted(all_special_chars)}")

print("\n\n컬럼별 특수문자 분포:")
for col, chars in special_char_by_column.items():
    if chars:  # 특수문자가 있는 컬럼만
        print(f"\n{col}: {sorted(chars)}")

발견된 특수문자 (총 0개):
sorted(all_special_chars): []


컬럼별 특수문자 분포:


### 특이한 특수문자들 일반 특수문자로 대체하기

In [12]:
# 특수문자 대체
# 형식: {'기존문자': '새로운문자'}

replacement_rules = {
    '¿': '?',
    '▣': '-',
    '□': '-',
    '▲': '-',
    '△': '-',
    '▶': '-',
    '◆': '-',
    '◇': '-',
    '○': '-',
    '●': '-',
    '–': '-',      # 엔 대시를 하이픈으로
    '—': '-',      # 이중 대시를 하이픈으로
    '―': '-',      # 수평 대시를 하이픈으로
    '─': '-',
    '－': '-',
    '＿': '-',
    '∼': '~',
    '∽': '~',
    '～': '~',
    '．': '.',
    '。': '.', 
    '：': ':',
    '…': '...',    # 생략 부호를 마침표 3개로
    '，': ',',
    '◦': '',
    '℃': '˚C',
    '°': '˚',
    '㎞': 'km',   # 킬로미터 단위
    '㎏': 'kg',
    '∞': 'infinite',
    '①': '1',     # 원형 숫자 1
    '＆': '&',     # 전각 앰퍼샌드를 일반 앰퍼샌드로
    '×': 'x',     # 곱하기 기호를 소문자 x로
    '＝': '=',
    '＋': '+',
    '√': 'sqrt ',
    '＜': '<',
    '＞': '>',
    '〈': '<',     # 왼쪽 꺽쇠 괄호
    '〉': '>',     # 오른쪽 꺽쇠 괄
    '「': '<',    # 왼쪽 일본식 큰따옴표
    '」': '>',    # 오른쪽 일본식 큰따옴표
    '｜': '|',
    '│':"|",
    '┃': "|",
    '“': "[",      # 좌측 큰따옴표를 작은따옴표로
    '”': "]",      # 우측 큰따옴표를 작은따옴표로
    '‘': "[",      
    '’': "]",
    '´': "'",
    '≪': "[",
    '≫': "]",
    '《': "[",
    '》': "]", 
    '『': "[",
    '』': "]", 
    '【': "[",
    '】': "]",
    '{': '[',
    '}': ']',
    '｢': '[',
    '｣': ']', 
    '（': '[',
    '）': ']',
    '\xad': '',    # 소프트 하이픈 제거
    '•': '',  # 다양한 점 문자 제거
    '·': '',  # 다양한 점 문자 제거
    '‧': '',  # 다양한 점 문자 제거
    '˙': '',
    '★': '',   # 제거할 문자들
    '☆': '',
    '♡': '',
    '♥': '',
    '♪': '',
    '♬': '',
    'ⓒ': '',
    '®': '',
    'ⓔ': '',
    'ⓘ': '',
    '†': '',
    '⚫': '',
    '◎': '',
    '↑': '',
    '→': '',
    '↓': '',
    '⇔': '',
    '≠': '!=',
    '└': '',
    '┗': '',
    '＊': '',
    '※': '',
    '*': '',
    '⊃': '',
    '÷': '나누기',
    '˚': '도',
    '∨': '',
    '≒': '',
    '≤': '<=',
    '≥': '>=',
    '■': '',
    '^^': '',
    '㈜': ''
    # 필요한 대로 더 추가할 수 있습니다
}

# 책소개 컬럼에 대체 적용
# df['책소개'] = df['책소개'].apply(lambda x: replace_chars(x, replacement_rules))

file_path = os.path.join(base_path, 'combined.csv')
df = pd.read_csv(file_path, encoding='utf-8')
text_columns = ['저자/아티스트', '출판사/제작사', '책소개', '목차']
for col in text_columns:
    if col in df.columns:
        df[col] = df[col].apply(lambda x: replace_chars(x, replacement_rules))

print("문자 대체 완료!")


# --- 선택한 단어 정제 ---
print("선택한 단어 정제 시작...")

patterns = [ r'지음']
df['출판사/제작사'] = df['출판사/제작사'].apply(
    lambda x: clean_text(x, patterns)
)

patterns = [r'지음', r'옮김', r'그림', r'외', r'공저', r'공역', r'편저', r'엮음', r'글', r'감수']
df['저자/아티스트'] = df['저자/아티스트'].apply(
    lambda x: clean_text(x, patterns)
)

patterns = [r'책소개', r'미리보기']
df['책소개'] = df['책소개'].apply(
    lambda x: clean_text(x, patterns)
)   

patterns = [r'목차', r'\.{2,}', r'\_{2,}']
df['목차'] = df['목차'].apply(
    lambda x: clean_text(x, patterns)
)

output_path = os.path.join(base_path, 'combined_preprocessed.csv')
df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"최종 전처리된 데이터가 '{output_path}'에 저장되었습니다.")

문자 대체 완료!
선택한 단어 정제 시작...
최종 전처리된 데이터가 './data/combined_preprocessed.csv'에 저장되었습니다.
